In [1]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 7.8 MB/s 
     |████████████████████████████████| 233 kB 48.1 MB/s 
     |████████████████████████████████| 1.4 MB 44.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=1e2ce1e05c48b585d6949f184d0af72dad5f53853fd33ba0e43792226996a3dc
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154950 sha256=da7dc9131b9087b0567c18043829bd4670f6f73077bfd8ab689cfb1d3eee0837
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [65]:
import pandas as pd

df_train = pd.read_csv('train.txt', sep="\t", on_bad_lines='skip',header=None)
df_train.columns = ["sent", "mood"]

In [14]:
print(df_train.head(10))

                                                sent   mood
0  خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...    SAD
1     از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم   HATE
2   "کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم...    SAD
3  چون همش با دوربین ثبت شده ، ایا میشه اعتراض زد...  OTHER
4              اين وضع ب طرز خنده داري گريه داره ...    SAD
5  خب من رسما از یک نفر متنفرم،چون از گربه بدش می...   HATE
6  داشتم خواب میدیدم یهو توی خواب گفتم بیاین درین...   FEAR
7               اصلا ارزش خرید نداره فقط رو مخ میره"  ANGRY
8  اونجایه آدمه مفعول ودروغگو...عجب خرایی داریم و...  ANGRY
9  من این گوشی رو اعصابم بهم ریخت کوبیدم تودیوار ...  HAPPY


In [66]:
import re

def remove_emojis_punctuations(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = re.sub(emoj, '', text)
    punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟'
    text = text.translate(str.maketrans(punctuations, ' '*len(punctuations)))
    return text

In [35]:
from hazm import *
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

In [67]:
def preprocess_text(data_frame):
  for index, row in data_frame.iterrows():
      if index%10 == 0:
        print(row['sent'])
      row['sent'] = remove_emojis_punctuations(row['sent'])
      row['sent'] = normalizer.normalize(row['sent'])
      for sent in sent_tokenize(row['sent']):
        for word in word_tokenize(row['sent']):
          if re.search(r'[a-zA-Z]', word):
            row['sent'] = row['sent'].replace(word, "")
          else:
            # new_word = stemmer.stem(word)
            new_word = lemmatizer.lemmatize(word)
            if new_word.__contains__("#"):
              new_word = new_word.split("#")[0]
            
            row['sent'] = row['sent'].replace(word, new_word)
      if index%10 == 0:
        print(row['sent'])
        print("-----------------------------")

In [ ]:
preprocess_text(df_train)

In [ ]:
#TEST data
df_test = pd.read_csv('test.txt', sep="\t", on_bad_lines='skip',header=None)
df_test.columns = ["sent", "mood"]
preprocess_text(df_test)

In [94]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
train_y = Encoder.fit_transform(df_train["mood"])
test_y = Encoder.fit_transform(df_test["mood"])

In [95]:
# TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(df_train['sent'].append(df_test['sent']))
train_X_Tfidf = tfidf_vect.transform(df_train['sent'])
test_X_Tfidf = tfidf_vect.transform(df_test['sent'])


In [99]:
# NB classifier

naive = naive_bayes.MultinomialNB()
naive.fit(train_X_Tfidf,train_y)

predictions_nb = naive.predict(test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_nb, test_y)*100)
print(classification_report(test_y, predictions_nb))

Naive Bayes Accuracy Score ->  20.677671589921808
              precision    recall  f1-score   support

           0       0.75      0.06      0.11       154
           1       1.00      0.04      0.07        57
           2       1.00      0.00      0.01       275
           3       0.00      0.00      0.00        65
           4       0.17      0.99      0.30       193
           5       0.89      0.13      0.23       262
           6       0.00      0.00      0.00       145

    accuracy                           0.21      1151
   macro avg       0.55      0.17      0.10      1151
weighted avg       0.62      0.21      0.12      1151



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [102]:
# SVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svm = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm.fit(train_X_Tfidf,train_y)
predictions_svm = svm.predict(test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_svm, test_y)*100)
print(classification_report(test_y, predictions_svm))

SVM Accuracy Score ->  41.09470026064292
              precision    recall  f1-score   support

           0       0.48      0.34      0.40       154
           1       0.70      0.56      0.62        57
           2       0.68      0.20      0.31       275
           3       0.33      0.12      0.18        65
           4       0.25      0.82      0.39       193
           5       0.63      0.55      0.59       262
           6       0.62      0.16      0.25       145

    accuracy                           0.41      1151
   macro avg       0.53      0.39      0.39      1151
weighted avg       0.54      0.41      0.40      1151



In [ ]:
# SVM with GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'degree' : [2,3,4],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']}
 
grid_svm = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2,cv = 2, n_jobs=-1)
grid_svm.fit(train_X_Tfidf,train_y)
grid_svm_predictions = grid_svm.predict(test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(grid_svm_predictions, test_y)*100)
print(classification_report(test_y, grid_svm_predictions))
print(grid_svm.best_params_)

In [105]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn = knn.fit(train_X_Tfidf, train_y)
predicted_knn = knn.predict(test_X_Tfidf)

print("KNN Accuracy Score -> ",accuracy_score(predicted_knn, test_y)*100)
print(classification_report(test_y, predicted_knn))

KNN Accuracy Score ->  34.231103388357944
              precision    recall  f1-score   support

           0       0.35      0.15      0.21       154
           1       0.34      0.40      0.37        57
           2       0.43      0.56      0.48       275
           3       0.11      0.25      0.15        65
           4       0.31      0.32      0.31       193
           5       0.42      0.29      0.35       262
           6       0.31      0.28      0.29       145

    accuracy                           0.34      1151
   macro avg       0.32      0.32      0.31      1151
weighted avg       0.36      0.34      0.34      1151



In [106]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree = decision_tree.fit(train_X_Tfidf,train_y)
predictions_decision_tree = decision_tree.predict(test_X_Tfidf)

print("Decision Tree Accuracy Score -> ",accuracy_score(predictions_decision_tree, test_y)*100)
print(classification_report(test_y, predictions_decision_tree))

Decision Tree Accuracy Score ->  35.186794092093834
              precision    recall  f1-score   support

           0       0.31      0.27      0.29       154
           1       0.51      0.56      0.53        57
           2       0.43      0.19      0.27       275
           3       0.34      0.23      0.28        65
           4       0.25      0.61      0.36       193
           5       0.44      0.40      0.42       262
           6       0.47      0.27      0.34       145

    accuracy                           0.35      1151
   macro avg       0.39      0.36      0.36      1151
weighted avg       0.39      0.35      0.34      1151



In [88]:
# Decision Tree with GridSearchCV

params = {'criterion': ['entropy', 'gini'], 'max_depth': [4,8,20,30,40,70,90,150]}
grid_decision_tree = GridSearchCV(DecisionTreeClassifier(), params, verbose=2, cv=3,n_jobs=-1)
grid_decision_tree = grid_decision_tree.fit(train_X_Tfidf,train_y)
predictions_grid_decision_tree = grid_decision_tree.predict(test_X_Tfidf)
print("Decision Tree with GridSearchCV Accuracy Score -> ",accuracy_score(predictions_grid_decision_tree, test_y)*100)
print(classification_report(test_y, predictions_grid_decision_tree))
print(grid_decision_tree.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Decision Tree2 Accuracy Score ->  35.79496090356212


In [107]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100)
random_forest = random_forest.fit(train_X_Tfidf,train_y)
predictions_random_forest = random_forest_clf.predict(test_X_Tfidf)

print("Random Forest Accuracy Score -> ",accuracy_score(y_pred, test_y)*100)
print(classification_report(test_y, predictions_random_forest))

Random Forest Accuracy Score ->  35.79496090356212
              precision    recall  f1-score   support

           0       0.48      0.21      0.29       154
           1       0.60      0.60      0.60        57
           2       0.60      0.14      0.22       275
           3       0.35      0.14      0.20        65
           4       0.22      0.92      0.36       193
           5       0.79      0.31      0.44       262
           6       0.65      0.19      0.30       145

    accuracy                           0.35      1151
   macro avg       0.53      0.36      0.34      1151
weighted avg       0.56      0.35      0.33      1151



In [108]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

gradient_boosting = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_depth=2, random_state=0)
gradient_boosting = gradient_boosting.fit(train_X_Tfidf,train_y)
predictions_gradient_boosting = gradient_boosting.predict(test_X_Tfidf)

print("Gradient Boosting Accuracy Score -> ",accuracy_score(predictions_gradient_boosting, test_y)*100)
print(classification_report(test_y, predictions_gradient_boosting))

Gradient Boosting Accuracy Score ->  16.85490877497828


In [ ]:
# Gradient Boosting with GridSearchCV
params = {'n_estimators': [10,20,30], 'max_depth': [2,10,20,30,70,150], 'random_state' : [0,5,10]}
grid_gradient_boosting = GridSearchCV(GradientBoostingClassifier(), params, verbose=2, cv=2,n_jobs=-1)
grid_gradient_boosting = grid_gradient_boosting.fit(train_X_Tfidf,train_y)
predictions_grid_gradient_boosting = grid_gradient_boosting.predict(test_X_Tfidf)
print("Decision Tree with GridSearchCV Accuracy Score -> ",accuracy_score(predictions_grid_gradient_boosting, test_y)*100)
print(classification_report(test_y, predictions_grid_gradient_boosting))

print(grid_gradient_boosting.best_params_)

In [120]:
# XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=100, n_jobs=-1,max_depth = 10)
xgb.fit(train_X_Tfidf,train_y)
predictions_xgb = xgb.predict(test_X_Tfidf)

print("XGBoost Accuracy Score -> ",accuracy_score(predictions_xgb, test_y)*100)
print(classification_report(test_y, predictions_xgb))

XGBoost Accuracy Score ->  40.660295395308424
              precision    recall  f1-score   support

           0       0.53      0.31      0.39       154
           1       0.63      0.72      0.67        57
           2       0.59      0.22      0.32       275
           3       0.47      0.22      0.29        65
           4       0.25      0.78      0.38       193
           5       0.59      0.46      0.51       262
           6       0.62      0.24      0.35       145

    accuracy                           0.41      1151
   macro avg       0.52      0.42      0.42      1151
weighted avg       0.52      0.41      0.40      1151



In [ ]:
# XGBoost with GridSearchCV
parameters = {
    "max_depth":[2, 10, 6, 12],
    "learning_rate":[0.3, 0.1, 0.03],
    "n_estimators":[20,100],
    "n_jobs" : [-1]}

grid_xgb = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = -1,
    cv = 2,
    verbose=True
)
grid_xgb = grid_xgb.fit(train_X_Tfidf,train_y)

print(model_gs.best_params_)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
